In [1]:
import requests

API_KEY = '1f011a9816a14e99b425e49761a7bfec'  
url = f'https://newsapi.org/v2/sources?apiKey={API_KEY}'

response = requests.get(url)

#Getting list of all sources to compare to allsides
if response.status_code == 200:
    sources = response.json()
    for source in sources['sources']:
        print(f"ID: {source['id']}, Name: {source['name']}, Category: {source['category']}")
else:
    print(f"Error: {response.status_code} - {response.text}")


ID: abc-news, Name: ABC News, Category: general
ID: abc-news-au, Name: ABC News (AU), Category: general
ID: aftenposten, Name: Aftenposten, Category: general
ID: al-jazeera-english, Name: Al Jazeera English, Category: general
ID: ansa, Name: ANSA.it, Category: general
ID: argaam, Name: Argaam, Category: business
ID: ars-technica, Name: Ars Technica, Category: technology
ID: ary-news, Name: Ary News, Category: general
ID: associated-press, Name: Associated Press, Category: general
ID: australian-financial-review, Name: Australian Financial Review, Category: business
ID: axios, Name: Axios, Category: general
ID: bbc-news, Name: BBC News, Category: general
ID: bbc-sport, Name: BBC Sport, Category: sports
ID: bild, Name: Bild, Category: general
ID: blasting-news-br, Name: Blasting News (BR), Category: general
ID: bleacher-report, Name: Bleacher Report, Category: sports
ID: bloomberg, Name: Bloomberg, Category: business
ID: breitbart-news, Name: Breitbart News, Category: general
ID: busines

In [2]:
import requests
import pandas as pd
from newspaper import Article
from fpdf import FPDF
import re

#API
API_KEY = '1f011a9816a14e99b425e49761a7bfec' 

#Query and Source (changed for liberal and conservative channels based on allsides designation)
query = 'voter'
sources = 'the-american-conservative'  

#Request URL
url = f'https://newsapi.org/v2/everything?q={query}&sources={sources}&apiKey={API_KEY}'

response = requests.get(url)

#Status
if response.status_code == 200:
    articles = response.json()  
    articles_df = pd.json_normalize(articles['articles'])

    #Number of Articles
    num_articles = len(articles_df)
    print(f"Number of articles found: {num_articles}\n")

    #Title and URL
    for index, doc in articles_df.iterrows():
        title = doc['title']
        url = doc['url']
        print(f"Title: {title}\nURL: {url}\n")

        #Download PDF
        try:
            article = Article(url)
            article.download()
            article.parse()

            sanitized_title = re.sub(r'[<>:"/\\|?*]', '', title)  # Remove invalid filename characters
            pdf_file_name = f"{sanitized_title[:50].replace(' ', '_')}.pdf"  # Limit title length for filename

            pdf = FPDF()
            pdf.add_page()
            
            pdf.set_font("Arial", 'B', 16)
            pdf.multi_cell(0, 10, title.encode('latin-1', 'replace').decode('latin-1'))  # Encode to handle special chars
            
            pdf.set_font("Arial", size=12)
            #Reemove random characters
            safe_text = article.text.encode('latin-1', 'replace').decode('latin-1')
            pdf.multi_cell(0, 10, safe_text)

            pdf.output(pdf_file_name)
            print(f"Saved: {pdf_file_name}\n")

        except Exception as e:
            print(f"Failed to download or save article '{title}': {e}")

else:
    print(f"Error: {response.status_code} - {response.text}")


Number of articles found: 7

Title: The Democrats’ Three-Decade War on Honest Voting
URL: https://www.theamericanconservative.com/the-democrats-three-decade-war-on-honest-voting/

Saved: The_Democrats’_Three-Decade_War_on_Honest_Voting.pdf

Title: Taylor Swift’s ‘Errors’ Tour?
URL: https://www.theamericanconservative.com/taylor-swifts-errors-tour/

Saved: Taylor_Swift’s_‘Errors’_Tour.pdf

Title: Will Speaker Johnson Eat His Words on a December Omnibus?
URL: https://www.theamericanconservative.com/will-speaker-johnson-eat-his-words-on-a-december-omnibus/

Saved: Will_Speaker_Johnson_Eat_His_Words_on_a_December_O.pdf

Title: Democrats Are a Major Threat to Free Speech; Trump Isn’t
URL: https://www.theamericanconservative.com/democrats-are-a-major-threat-to-free-speech-trump-isnt/

Saved: Democrats_Are_a_Major_Threat_to_Free_Speech;_Trump.pdf

Title: Trump Ripens in the Sun Belt
URL: https://www.theamericanconservative.com/trump-ripens-in-the-sun-belt/

Saved: Trump_Ripens_in_the_Sun_Belt